In [1]:
# Standard block to equalize local and Colab.
from pathlib import Path
import os
try:
    # See if we are running on google.colab
    import google.colab
    from google.colab import files
    # Configure kaggle
    files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
    !pip install -q kaggle
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    # Download the workshop repo and change to its directory
    # For now edit the username/password. This requirement will be removed when the repo is made public.
    !git clone https://<username>:<password>@github.com/SachsLab/IntracranialNeurophysDL.git ../repo
    os.chdir('../repo')
    !pip install tensorboardcolab
    from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
    tbc=TensorBoardColab(startup_waiting_time=30)
    tensorboard_callback = TensorBoardColabCallback(tbc)
    IN_COLAB = True
except ModuleNotFoundError:
    import sys
    if Path.cwd().stem == 'notebooks':
        os.chdir(Path.cwd().parent)
    os.environ['PATH'] = os.environ['PATH'] + ';' + str(Path(sys.executable).parent / 'Scripts')
    import datetime
    import tensorflow as tf
    %load_ext tensorboard
    # Clear any logs from previous runs
    !rm -rf ./logs/ 
    log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    IN_COLAB = False
    

In [2]:
# Download faces_basic if we don't already have it.
# If you get 401 - Unauthorized, then you need to create a new API token from kaggle:

datadir = Path.cwd() / 'data' / 'kjm_ecog'
if not (datadir / 'converted').is_dir():
    !kaggle datasets download -d cboulay/kjm-ecog-faces-basic
    print("Finished downloading. Now extracting contents...")
    data_path = Path('kjm-ecog-faces-basic.zip')
    import zipfile
    with zipfile.ZipFile(data_path, 'r') as zip_ref:
        zip_ref.extractall(datadir / 'converted' / 'faces_basic')
    data_path.unlink()


kjm-ecog-faces-basic.zip: Skipping, found more recently modified local copy (use --force to force download)
Finished downloading. Now extracting contents...


In [3]:
# Load data from one participant.
SUB_ID = 'de'
from data.utils.fileio import from_neuropype_h5
test_file = datadir / 'converted' / 'faces_basic' / (SUB_ID + '_bp.h5')
chunks = from_neuropype_h5(test_file)
chunk_names = [_[0] for _ in chunks]
chunk = chunks[chunk_names.index('signals')][1]
ax_types = [_['type'] for _ in chunk['axes']]
instance_axis = chunk['axes'][ax_types.index('instance')]
n_trials = len(instance_axis['data'])
X = chunk['data'].reshape((n_trials, -1))  # 603 trials x 527 features
Y = instance_axis['data']['Marker'].values
n_trials, n_features = X.shape
print(X.shape, Y.shape)


(603, 527) (603,)


## Creating an artificial neural net with Tensorflow and Keras

We will next create a "model". The model maps the input data to some output.

Each trial _idx_ in our input has our brain signal data X[idx] and a label Y[idx].

Each data point in X[idx] is of shape (n_features,),
where n_features is the product m_channels * n_samples.

Each label (Y[idx]) is 1 of 3 values: "ISI", "face", or "house".

So we want to find a model that maps from an n_features input to 3 outputs,
where each of the outputs is a score associated with a class.
We will use a simple linear fully-connected model with a single layer.
I honestly couldn't think of a simpler neural network.

![simple linear network](img/simple_linear_nn.png) 
created with [NN-SVG](http://alexlenail.me/NN-SVG/index.html)

We use Tensorflow to create and train the model.
The model specification is done using the [Keras API, as provided by tensorflow](https://www.tensorflow.org/guide/keras).

Tensorflow has ["eager execution"](https://www.tensorflow.org/guide/eager)
enabled so TensorFlow operations execute immediately and their results can be inspected.
This is the default behaviour in upcoming TensorFlow 2.0 so we might as well get used to it.
[Another tf.eager tutorial here.](https://adventuresinmachinelearning.com/tensorflow-eager-tutorial/)



In [4]:
import tensorflow as tf

tf.enable_eager_execution()

# Create the model.
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(3, input_shape=(n_features,), activation='linear'))
model.add(tf.keras.layers.Softmax())  # Convert to probability scores, summing to 1


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 1584      
_________________________________________________________________
softmax (Softmax)            (None, 3)                 0         
Total params: 1,584
Trainable params: 1,584
Non-trainable params: 0
_________________________________________________________________


Before the network is ready, it needs three more things:
* A loss function. See options in tf.losses
* An optimizer. See options in tf.keras.optimizers ('Adam', 'rmsprop', 'SGD')
* Metrics to monitor during training

[neat graph for optimizers](https://www.tensorflow.org/tutorials/eager/custom_training_walkthrough#create_an_optimizer)

These are configured during compilation

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()


## Prepare data to feed into the model during training
Tensorflow has a data API called [tf.data](https://www.tensorflow.org/guide/datasets).

[tf.data tutorial](https://adventuresinmachinelearning.com/tensorflow-dataset-tutorial/)

TODO: Explain one-hot encoding.
TODO: Train and validation.
TODO: Batch size


In [ ]:
BATCH_SIZE = 5

# Import our data as a Dataset.
y = tf.reshape(tf.one_hot(Y, 3), (-1, 3))  # y = tf.keras.utils.to_categorical(Y)
dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(1000).repeat().batch(BATCH_SIZE)
ds_iter = dataset.make_initializable_iterator()
next_element = ds_iter.get_next()

## Train the model
TODO: epochs

In [ ]:
# model.fit(x=x_train, 
#           y=y_train, 
#           epochs=5, 
#           validation_data=(x_test, y_test), 
#           callbacks=[tensorboard_callback])
